# RandomizedSearchCV
RandomizedSearchCV 是一个可以代替 GridSearchCV的工具。通常，相比Grid Search，Randomized Search效率会更高。具体的原理如下：
1. 对于搜索范围是分布的超参数，根据给定的分布随机采样；
2. 对于搜索范围是list的超参数，在给定的list中进行等概率采样；
3. 对前面两步中得到的n_iter组采样结果，进行遍历。
4. 如果给定的搜索范围均为list，则不放回抽样n_iter次。
5. 在sklearn中，交叉验证随机搜索的类为sklearn.model_selection.RandomizedSearchCV

关键参数如下：
- estimator: 模型
- n_iter: 迭代次数查找次数
- n_jobs: 并行数，设置为-1时，则用所有的处理器
- cv: 交叉验证折数，不给则默认为5
- error_score: 如果error了，记多少分。默认是'raise'，也就是报错，也可以选择一个分数，例如-1

# 基于XGB的随机搜索

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
'''
用于设置RandomizedSearchCV中模型的分布
例如：查找连续值使用 uniform
     uniform(1,5), 指的是从1-5间等可能选择浮点数; 
     查找整数值可以使用randint
     randint(1,5), 指的是从1-5间等可能选择整数
'''
from scipy.stats import uniform, randint

from xgboost import XGBRFClassifier
from sklearn.datasets import load_iris

iris = load_iris()

X = iris.data
y = iris.target
'''
n_estimators: 从500-5000等可能选择整数
learning_rate: 从这5个值中等可能地选择
subsample: 从0.3-0.9等可能选择浮点数
colsample_bytree: 从0.5-0.9等可能选择浮点数
'''
# params = {'n_estimators' : randint(500,5000), 
#           'learning_rate': [0.001, 0.01, 0.1, 0.2, 0, 3], 
#           'subsample': uniform(0.3, 0.9), 
#           'colsample_bytree': uniform(0.5, 0.9)
#           }
params = {'n_estimators' : [i for i in range(50, 501, 50)], 
          'learning_rate': [0.001, 0.01, 0.1, 0.2, 0, 3], 
          'subsample': [i / 10 for i in range(3, 10)], 
          'colsample_bytree': [i / 10 for i in range(5, 9)]
          }

xgb = XGBRFClassifier(objective='multi:softmax')
'''
使用xgb为estimator
参数列表为 params
n_iter:迭代次数1000
评判标准为 accuracy
注意：二分类时可以使用 scoring 可以设置为 roc_auc,多分类需要用 accuracy
如果error了, 记为0分
用所有的处理器进行计算
'''
clf = RandomizedSearchCV(xgb, 
                         param_distributions = params, 
                         n_iter = 200, 
                         scoring = 'accuracy', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)

# 开始搜索， 返回值中的model的参数是假的，看best estimator使用下面的命令
clf.fit(X, y)

# 查看最好的预测器
print(clf.best_estimator_)

# 输出最优参数
print(clf.best_params_)

# 基于深度学习的随机搜索

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt 
%matplotlib inline    
#为了能在notebook中显示图像
import numpy as np
import sklearn   
import pandas as pd 
import os 
import sys 
import time 
import tensorflow as tf 
from tensorflow import keras 


In [ ]:
from sklearn.datasets import fetch_california_housing #从sklearn中引用加州的房价数据

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

In [ ]:
#引用train_test_split对数据集进行拆分
# test_size 控制切分比例，默认切分比例3:1
from sklearn.model_selection import train_test_split  

#拆分数据集，加载数据集后返回训练集以及测试集
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state = 1) 

#将训练集进行一次拆分为验证集和测试集
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=2)

print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#对数据进行归一化处理

#由于transform处理处理数据时二维数组，所以要将数据转化一下
#x_train: [none, 28, 28] -> [none, 784]
#对于使用fit_transform 和transform 请参考我的TensorFlow中的博客
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

#注意在归一化数据后，之后使用的数据要使用新的归一化数据

In [ ]:
# 以下展示如何实现随机超参数搜索

# 1.将tf.keras.models。sequential转化为sklearn的model

# 封装模型
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal


def bulid_model(hidden_layers=1, layer_size=30, learning_rate=3e-3):

    # 使用序贯模型Sequential   tf.keras.models.sequential()
    model = keras.models.Sequential()
    # 第一个layer不循环创建，因为要输入input_shape,之后的layer可以循环创建
    model.add(keras.layers.Dense(
        layer_size, activation="relu", input_shape=x_train.shape[1:]))
    for _ in range(hidden_layers - 1):
        model.add(keras.layers.Dense(layer_size, activation="relu"))
    model.add(keras.layers.Dense(1))
    # 定义优化函数，使用自定义的learning_rate
    optimizer = keras.optimizers.Adam(learning_rate)
    # 编译compile
    model.compile(loss="mean_squared_error",  # 损失函数：使用均方根误差
                  optimizer=optimizer,  # 优化函数
                  )
    return model


# 转化模型  tf.keras.wrappers.scikit_learn.KerasRegressor
sklearn_model = keras.wrappers.scikit_learn.KerasRegressor(bulid_model)

# 2.定义要调整的超参数集合
# scipy中的reciprocal函数用于实现分布，通过这个分布来实现学习率的超参数获取

param_distribution = {
    "hidden_layers": [1, 2, 3, 4],
    "layer_size": np.arange(10, 100, 10),
    "learning_rate": reciprocal(1e-4, 1e-2)
}

# 3.参数搜索 RandomizedSearchCV


# RandomizedSearchCV参数说明，
# clf1设置训练的学习器
# param_dist字典类型，放入参数搜索范围
# scoring = 'neg_log_loss'，精度评价方式设定为"neg_log_loss"
# n_iter=300，训练300次，数值越大，获得的参数精度越大，但是搜索时间越长
# n_jobs = -1，使用所有的CPU进行训练，默认为1，使用1个CPU
# RandomizedSearchCV采用了cross-validation: 将训练集分成n分，n-1训练，最后一份验证。默认cv=3
random_search_cv = RandomizedSearchCV(sklearn_model,
                                      param_distribution,
                                      cv=3,
                                      n_iter=5,
                                      n_jobs=1)
# 使用回调函数
callbacks = [
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
# 训练函数
random_search_cv.fit(x_train_scaled, y_train,
                     epochs=100,
                     validation_data=(x_valid_scaled, y_valid),
                     callbacks=callbacks)

print(random_search_cv.best_params_)
print(random_search_cv.best_score_)
print(random_search_cv.best_estimator_)
